In [ ]:
# Let's start with some lovely imports that should have been installed if not available by default
import json
import requests
from zenroom import zenroom


In [ ]:
# Test zenroom is correctly installed and running witht the following function
def generate_random_challenge():
    """
        This function calls zenroom to generate
        a random string to be used as challenge
    """
    contract = """
        rule check version 1.0.0
        Given nothing
        When I create the random object of '512' bits
        and I rename the 'random_object' to 'challenge'
        Then print 'challenge'
    """

    try:
        result = zenroom.zencode_exec(contract, conf='debug=0')
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    res_json = json.loads(result.output)

    print(f"Generated challenge: {res_json['challenge']}")

    return


In [ ]:
# This should print something like
# Generated challenge: MMQ1JSrCA7L4QNftLyaaSRunT4Z9+Rr2QkE+a+DWLEljtg6EroLbCj5VjLH+xba9Rv1D+3ncQHw5s/lH41IFJw==
generate_random_challenge()

In [ ]:
# Define user data that is going to be user in the GraphQL calls
user1_data = {
  "userChallenges": {
    "whereParentsMet": "Manicomio",
    "nameFirstPet": "Godzilla",
    "nameFirstTeacher": "JimiHendrix",
    "whereHomeTown": "Giuddili",
    "nameMotherMaid": "Promiscua"
  },
  "name": "User One",
  "username": "userone_username",
  "email": "userone@example.org",
  "lat": 52.35871773455108,
  "long": 4.916762398221842,
  "addr": "Oosterpark 9, 1091 AC Amsterdam",
  "note": "me.userone.org"
}

user2_data = {
    "userChallenges": {
        "whereParentsMet":"OntheMoon",
        "nameFirstPet":"Lucifer",
        "nameFirstTeacher":"Satan",
        "whereHomeTown":"Hell",
        "nameMotherMaid":"Theresa"
    },
    "name": "User Two",
    "username": "usertwo",
    "email": "usertwo@example.org",
    "lat" : 51.47240440868687,
    "long" : 5.412460440524406,
    "addr" : "De schakel 30, 5651 Eindhoven",
    "note" : "me.usertwo.org"
}


In [ ]:
# What endpoint are we talking to?
endpoint = 'http://65.109.11.42:9000/api'

In [ ]:
# Get the seed from the server to later generate the keypair
def get_HMAC(email):

    variables = {
        "firstRegistration": True,
        "userData": "{\"email\": \"" + email + "\"}"
    };

    
    payload = {
      "query": """mutation ($firstRegistration: Boolean!, $userData: String!){
  
        keypairoomServer(firstRegistration: $firstRegistration, userData: $userData)
      
      }""",
      "variables": variables
    }
    

    r = requests.post(endpoint, json=payload)
    
    result = r.json()

    print(json.dumps(result, indent=2))
    
    return result

In [ ]:
res = get_HMAC(user1_data['email'])
# save the HMAC in the user data
user1_data['seedServerSideShard.HMAC'] = res['data']['keypairoomServer']

In [ ]:
res = get_HMAC(user2_data['email'])
# save the HMAC in the user data
user2_data['seedServerSideShard.HMAC'] = res['data']['keypairoomServer']

In [ ]:
# Generate the user keypair (and the mnemonic seed)
def generate_keypair(userdata):
    """
        This function calls zenroom to generate
        a keypair using the server-provided HMAC
    """
    contract = """
        Scenario 'ecdh': Create the key
        Scenario 'ethereum': Create key
        Scenario 'reflow': Create the key
        Scenario 'schnorr': Create the key
        Scenario 'eddsa': Create the key
        Scenario 'qp': Create the key


        # Loading the user name from data
        Given my name is in a 'string' named 'username'

        # Loading the answers from 3 secret questions. The user will have to pick the 3 challenges from a list 
        # and have to remember the questions - the order is not important cause Zenroom will sort alphabetically 
        # the data in input
        #
        # NOTE: the challenges will never be communicated to the server or to anybody else!
        Given I have a 'string dictionary' named 'userChallenges'

        # Loading the individual challenges, in order to have them hashed 
        # and the hashes OPTIONALLY stored by the server, to improve regeneration of the keypair
        Given I have a 'string' named 'whereParentsMet' in 'userChallenges'
        Given I have a 'string' named 'nameFirstPet' in 'userChallenges'
        Given I have a 'string' named 'whereHomeTown' in 'userChallenges'
        Given I have a 'string' named 'nameFirstTeacher' in 'userChallenges'
        Given I have a 'string' named 'nameMotherMaid' in 'userChallenges'

        # Loading the pbkdf received from the server, containing a signed hash of known data
        Given that I have a 'base64' named 'seedServerSideShard.HMAC' 

        # Save the backup for mnemonic dump, before factoring with the salt
        # it is shortened to 16 bytes by hashing sha512 the KDF and taking the first 16 bytes
        When I create the key derivation of 'userChallenges'
        and I create the hash of 'key derivation' using 'sha512'
        and I split the leftmost '16' bytes of 'hash'
        and I delete the 'key derivation'
        and I delete the 'hash'
        and I rename the 'leftmost' to 'seed'

        # Hash again the user's challenges with salt for the seed root
        When I rename 'seedServerSideShard.HMAC' to 'salt'
        and I create the key derivation of 'seed' with password 'salt'
        and I rename the 'key derivation' to 'seed.root'

        # In the following flow the order should NOT be changed

        When I create the hash of 'seed.root'
        When I rename the 'hash' to 'seed.ecdh'

        When I create the hash of 'seed.ecdh'
        When I rename the 'hash' to 'seed.eddsa'

        When I create the hash of 'seed.eddsa'
        When I rename the 'hash' to 'seed.ethereum'

        When I create the hash of 'seed.ethereum'
        When I rename the 'hash' to 'seed.reflow'

        When I create the hash of 'seed.reflow'
        When I rename the 'hash' to 'seed.schnorr'

        # end of the sorted creation flow

        When I create the ecdh key with secret key 'seed.ecdh'
        When I create the eddsa key with secret key 'seed.eddsa'
        When I create the ethereum key with secret key 'seed.ethereum'
        When I create the reflow key with secret key 'seed.reflow'
        When I create the schnorr key with secret key 'seed.schnorr'

        When I create the ecdh public key
        When I create the eddsa public key
        When I create the ethereum address
        When I create the reflow public key
        When I create the schnorr public key

        # Creating the hashes of the single challenges, to OPTIONALLY help 
        # regeneration of the keypair

        When I create the 'base64 dictionary'
        and I rename the 'base64 dictionary' to 'hashedAnswers'

        When I create the key derivation of 'whereParentsMet'
        and I rename the 'key derivation' to 'whereParentsMet.kdf'
        When I insert 'whereParentsMet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstPet'
        and I rename the 'key derivation' to 'nameFirstPet.kdf'
        When I insert 'nameFirstPet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'whereHomeTown'
        and I rename the 'key derivation' to 'whereHomeTown.kdf'
        When I insert 'whereHomeTown.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstTeacher'
        and I rename the 'key derivation' to 'nameFirstTeacher.kdf'
        When I insert 'nameFirstTeacher.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameMotherMaid'
        and I rename the 'key derivation' to 'nameMotherMaid.kdf'
        When I insert 'nameMotherMaid.kdf' in 'hashedAnswers'


        # This prints the keyring
        Then print the 'keyring' 

        # this prints the hashes of the challenges
        # Then print the 'hashedAnswers'

        # This prints the seed for the private keys as mnemonic 
        Then print the 'seed' as 'mnemonic'

        Then print the 'ecdh public key'
        Then print the 'eddsa public key'
        Then print the 'ethereum address'
        Then print the 'reflow public key'
        Then print the 'schnorr public key'
    """
    
    data = json.dumps(userdata)

    try:
        result = zenroom.zencode_exec(contract, data=data)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

#     print(f'result: {result}')
    res_json = json.loads(result.output)

    print(f"Generated keypair data: {json.dumps(res_json, indent=2)}")

    return res_json


In [ ]:
res = generate_keypair(user1_data)

user1_data['seed'] = res['seed']
user1_data['eddsa_public_key'] = res['eddsa_public_key']
user1_data['keyring'] = {}
user1_data['keyring']['eddsa'] = res['keyring']['eddsa']


In [ ]:
res = generate_keypair(user2_data)

user2_data['seed'] = res['seed']
user2_data['eddsa_public_key'] = res['eddsa_public_key']
user2_data['keyring'] = {}
user2_data['keyring']['eddsa'] = res['keyring']['eddsa']


In [ ]:
# Temporary we need a key to create a person
with open('.credentials.json') as f:
    data = json.load(f)
    SECRET_KEY = data['key']

In [ ]:
# Create the person using their public key

def create_Person(name, username, email, eddsaPublicKey):

    variables = {
    "person": {
        "name": name,
        "user": username,
        "email": email,
        "eddsaPublicKey": eddsaPublicKey
        }
    }

    payload = {
      "query": """mutation ($person: PersonCreateParams!){
        createPerson(person: $person)
        {
            agent{
                id
                name
                user
                email
                eddsaPublicKey
            }
        }
       }""",
      "variables": json.dumps(variables)
    }

    print(json.dumps(payload, indent=2))
    r = requests.post(endpoint, json=payload, headers={'zenflows-admin': SECRET_KEY})
    
    result = r.json()

    print(json.dumps(result, indent=2))
    
    return result

In [ ]:
res = create_Person(user1_data['name'], user1_data['username'], user1_data['email'], user1_data['eddsa_public_key'])
user1_data['id'] = res['data']['createPerson']['agent']['id']
print(json.dumps(user1_data, indent=2))

In [ ]:
res = create_Person(user2_data['name'], user2_data['username'], user2_data['email'], user2_data['eddsa_public_key'])
user2_data['id'] = res['data']['createPerson']['agent']['id']
print(json.dumps(user2_data, indent=2))